In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("UDF")
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/13 16:35:43 INFO SparkContext: Running Spark version 3.5.5
25/03/13 16:35:43 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/13 16:35:43 INFO SparkContext: Java version 11.0.26
25/03/13 16:35:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


spark: SparkSession = org.apache.spark.sql.SparkSession@32b18bf3
import spark.implicits._

In [4]:
val data = Seq(
    (1, "Michael", 3000),
    (2, "Andy", 4500),
    (3, "Justin", 3500),
    (4, "Berta", 4000)
).toDF("id", "name", "salary")

data.show()

+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  1|Michael|  3000|
|  2|   Andy|  4500|
|  3| Justin|  3500|
|  4|  Berta|  4000|
+---+-------+------+



data: DataFrame = [id: int, name: string ... 1 more field]

## UDF без аргументов

In [5]:
val randomUDF = udf(() => java.lang.Math.random())
data.withColumn("random", randomUDF()).show()

+---+-------+------+-------------------+
| id|   name|salary|             random|
+---+-------+------+-------------------+
|  1|Michael|  3000| 0.6245355820477106|
|  2|   Andy|  4500| 0.2533622519032023|
|  3| Justin|  3500|0.11614977241890445|
|  4|  Berta|  4000| 0.7590926783068178|
+---+-------+------+-------------------+



randomUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd5$Helper$$Lambda$5018/0x0000000801977c40@6f8ae8ba,
  DoubleType,
  List(),
  Some(
    ExpressionEncoder(
      BoundReference(0, DoubleType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, DoubleType),
          DoubleType,
          List("- root class: \"double\"")
        ),
        List("- root class: \"double\"")
      ),
      Double
    )
  ),
  None,
  false,
  true
)

## UDF с одним аргументом

In [6]:
val plusOneUDF = udf((x: Int) => x + 1)
data.select($"name", $"salary", plusOneUDF($"salary").as("new_salary")).show()

+-------+------+----------+
|   name|salary|new_salary|
+-------+------+----------+
|Michael|  3000|      3001|
|   Andy|  4500|      4501|
| Justin|  3500|      3501|
|  Berta|  4000|      4001|
+-------+------+----------+



plusOneUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd6$Helper$$Lambda$5095/0x00000008019a8040@79b24a23,
  IntegerType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, IntegerType),
            IntegerType,
            List("- root class: \"int\"")
          ),
          List("- root class: \"int\"")
        ),
        Int
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, IntegerType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, IntegerType),
          IntegerType,
          List("- root class: \"int\"")
        ),
        List("- root class: \"int\"")
      ),
      Int
    )
  ),
  None,
  false,
  true
)

## UDF с двумя аргументами

In [7]:
val strLenPlusUDF = udf((x: String, y: Int) => x.length + y)

strLenPlusUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd7$Helper$$Lambda$5135/0x00000008019c6840@5ea2c689,
  IntegerType,
  List(
    Some(
      ExpressionEncoder(
        StaticInvoke(
          class org.apache.spark.unsafe.types.UTF8String,
          StringType,
          "fromString",
          List(BoundReference(0, ObjectType(class java.lang.String), true)),
          List(),
          true,
          false,
          true
        ),
        Invoke(
          UpCast(
            GetColumnByOrdinal(0, StringType),
            StringType,
            List("- root class: \"java.lang.String\"")
          ),
          "toString",
          ObjectType(class java.lang.String),
          List(),
          List(),
          true,
          false,
          true
        ),
        java.lang.String
      )
    ),
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetCo

In [8]:
data.select($"id", $"name", length($"name"), strLenPlusUDF($"name",  $"id").as("nameLenPlusId")).show()

+---+-------+------------+-------------+
| id|   name|length(name)|nameLenPlusId|
+---+-------+------------+-------------+
|  1|Michael|           7|            8|
|  2|   Andy|           4|            6|
|  3| Justin|           6|            9|
|  4|  Berta|           5|            9|
+---+-------+------------+-------------+



In [9]:
data.select($"name", length($"name"), strLenPlusUDF($"name", lit(1)).as("nameLenPlus1")).show()

+-------+------------+------------+
|   name|length(name)|nameLenPlus1|
+-------+------------+------------+
|Michael|           7|           8|
|   Andy|           4|           5|
| Justin|           6|           7|
|  Berta|           5|           6|
+-------+------------+------------+



## UDF для выражений WHERE

In [10]:
val oneArgFilterUDF= udf((n: Int) => { n > 2 })
data.where(oneArgFilterUDF($"id")).show()

+---+------+------+
| id|  name|salary|
+---+------+------+
|  3|Justin|  3500|
|  4| Berta|  4000|
+---+------+------+



oneArgFilterUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd10$Helper$$Lambda$5164/0x00000008019d2840@3f4cf745,
  BooleanType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, IntegerType),
            IntegerType,
            List("- root class: \"int\"")
          ),
          List("- root class: \"int\"")
        ),
        Int
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, BooleanType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, BooleanType),
          BooleanType,
          List("- root class: \"boolean\"")
        ),
        List("- root class: \"boolean\"")
      ),
      Boolean
    )
  ),
  None,
  false,
  true
)

## Spark SQL UDF

In [11]:
val cubed = (s: Long) => {s * s * s}

cubed: Long => Long = ammonite.$sess.cmd11$Helper$$Lambda$5179/0x00000008019dd840@7f57ec72

In [12]:
spark.udf.register("cubed", cubed)

res12: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd11$Helper$$Lambda$5179/0x00000008019dd840@7f57ec72,
  LongType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, LongType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, LongType),
            LongType,
            List("- root class: \"long\"")
          ),
          List("- root class: \"long\"")
        ),
        Long
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, LongType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, LongType),
          LongType,
          List("- root class: \"long\"")
        ),
        List("- root class: \"long\"")
      ),
      Long
    )
  ),
  Some("cubed"),
  false,
  true
)

In [13]:
spark.range(1, 10).createOrReplaceTempView("udf_test")

In [14]:
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

+---+--------+
| id|id_cubed|
+---+--------+
|  1|       1|
|  2|       8|
|  3|      27|
|  4|      64|
|  5|     125|
|  6|     216|
|  7|     343|
|  8|     512|
|  9|     729|
+---+--------+



## Производительность

In [15]:
val train = spark.read.option("inferSchema", true).option("header", true).csv("data/train.csv").cache()

train: Dataset[Row] = [row_id: int, timestamp: bigint ... 8 more fields]

In [16]:
train.show(5)

25/03/13 16:41:16 WARN MemoryStore: Not enough space to cache rdd_17_41 in memory! (computed 26.3 MiB so far)
25/03/13 16:41:16 WARN BlockManager: Persisting block rdd_17_41 to disk instead.
25/03/13 16:41:16 WARN MemoryStore: Not enough space to cache rdd_17_40 in memory! (computed 41.1 MiB so far)
25/03/13 16:41:16 WARN BlockManager: Persisting block rdd_17_40 to disk instead.
25/03/13 16:41:24 WARN MemoryStore: Not enough space to cache rdd_17_40 in memory! (computed 41.1 MiB so far)
25/03/13 16:41:32 WARN MemoryStore: Not enough space to cache rdd_17_41 in memory! (computed 26.3 MiB so far)


+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|row_id|timestamp|user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|     0|        0|    115|      5692|              0|                1|          3|                 1|                       NULL|                          NULL|
|     1|    56943|    115|      5716|              0|                2|          2|                 1|                    37000.0|                         false|
|     2|   118363|    115|       128|              0|                0|          0|                 1|                    55000.0|                         false|
|     3|   131167|    115|  

In [17]:
train.count()

25/03/13 16:41:40 WARN MemoryStore: Not enough space to cache rdd_17_41 in memory! (computed 7.7 MiB so far)
25/03/13 16:41:41 WARN MemoryStore: Not enough space to cache rdd_17_40 in memory! (computed 26.3 MiB so far)


res17: Long = 101230332L

In [18]:
val to_month = udf((ts_ms: Long) => ts_ms / 31536000000L * 12) // 1 year is 31536000000 ms

to_month: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd18$Helper$$Lambda$7635/0x0000000801dfc840@4b420a16,
  LongType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, LongType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, LongType),
            LongType,
            List("- root class: \"long\"")
          ),
          List("- root class: \"long\"")
        ),
        Long
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, LongType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, LongType),
          LongType,
          List("- root class: \"long\"")
        ),
        List("- root class: \"long\"")
      ),
      Long
    )
  ),
  None,
  false,
  true
)

In [ ]:
val t1 = System.nanoTime

train
    .select($"content_id", to_month($"timestamp").as("month"))
    .groupBy("content_id")
    .mean("month")
    .show(10)

val duration1 = (System.nanoTime - t1) / 1e9d

25/03/13 16:41:51 WARN MemoryStore: Not enough space to cache rdd_17_8 in memory! (computed 15.1 MiB so far)
25/03/13 16:41:55 WARN MemoryStore: Not enough space to cache rdd_17_20 in memory! (computed 41.2 MiB so far)


+----------+------------------+
|content_id|        avg(month)|
+----------+------------------+
|      4519|0.4460020768431983|
|      4818|0.5068813295247988|
|      5518|0.6536082474226804|
|     13285| 2.725925925925926|
|     12027|2.2696177062374248|
|       833|0.7334516865330967|
|      9427| 0.652203699474766|
|       496|0.6270834569072898|
|      5156|0.5233297393761386|
|      6336|0.6127002288329519|
|      2866|0.9061162849232318|
|      1959|1.0353606789250354|
|      7982|0.6966064981949458|
|     23336| 0.564428149870339|
|       148| 0.602506180206291|
|      1342|0.7789912124752375|
|      1088|0.7588338484671289|
|      1580|0.7253544345532477|
|       471|0.8491909385113269|
|      2122|0.8964813870474247|
+----------+------------------+
only showing top 20 rows



t1: Long = 23040926265458L
duration1: Double = 19.477131833

In [ ]:
val t2 = System.nanoTime

train
    .select($"content_id", ($"timestamp" / 31536000000L * 12).alias("month"))
    .groupBy("content_id")
    .mean("month")
    .show(10)

val duration2 = (System.nanoTime - t2) / 1e9d

25/03/13 16:42:07 WARN MemoryStore: Not enough space to cache rdd_17_10 in memory! (computed 26.3 MiB so far)
25/03/13 16:42:08 WARN MemoryStore: Not enough space to cache rdd_17_13 in memory! (computed 26.2 MiB so far)
25/03/13 16:42:09 WARN MemoryStore: Not enough space to cache rdd_17_14 in memory! (computed 26.3 MiB so far)
25/03/13 16:42:09 WARN MemoryStore: Not enough space to cache rdd_17_15 in memory! (computed 41.1 MiB so far)
25/03/13 16:42:12 WARN MemoryStore: Not enough space to cache rdd_17_20 in memory! (computed 41.2 MiB so far)


+----------+------------------+
|content_id|        avg(month)|
+----------+------------------+
|      4519|2.2218736221822732|
|      4818|2.3372134952525556|
|      5518| 3.029691836679273|
|     13285| 5.964229418381344|
|     12027|5.9773868010962765|
|       833|  3.25216382900438|
|      9427| 3.108803192689145|
|       496|2.9023706505861853|
|      5156|2.6012427887203198|
|      6336|2.7640873674707946|
|      2866|3.8766860651262456|
|      1959|3.8461177532795423|
|      7982|3.0568602514634398|
|     23336| 2.773484159471813|
|       148| 2.579153999078694|
|      1342|  3.16126199595755|
|      1088| 3.545207472523368|
|      1580|3.2343945900405027|
|       471| 3.760585843300882|
|      2122|3.7522109521798543|
+----------+------------------+
only showing top 20 rows



t2: Long = 23061690404833L
duration2: Double = 16.6974715

In [21]:
println(s"duration1 = $duration1\nduration2 = $duration2\nduration1 - duration2 = ${duration1 - duration2}")

duration1 = 19.477131833
duration2 = 16.6974715
duration1 - duration2 = 2.7796603330000025
